## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]

# Preview the new DataFrame
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
x_columns = [
    "Age",
    "DistanceFromHome",
    "Education",
    "EnvironmentSatisfaction",
    "JobInvolvement",
    "JobLevel",
    "YearsSinceLastPromotion",
    "BusinessTravel",
    "MaritalStatus",
    "OverTime"
]

# Create X_df using your selected columns
X_df = attrition_df[x_columns]

# Show the data types for X_df
X_df.dtypes


,0
Age,int64
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
JobInvolvement,int64
JobLevel,int64
YearsSinceLastPromotion,int64
BusinessTravel,object
MaritalStatus,object
OverTime,object


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# Define the features (X) and targets (y)
X = X_df
y = y_df

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Preview the shapes of the training and testing sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1102, 10), (368, 10), (1102, 2), (368, 2))

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary (OneHotEncoding)
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Check the updated data types for X_train
X_train.dtypes


,0
Age,int64
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
JobInvolvement,int64
JobLevel,int64
YearsSinceLastPromotion,int64
BusinessTravel_Travel_Frequently,bool
BusinessTravel_Travel_Rarely,bool
MaritalStatus_Married,bool


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [8]:
# Preview the scaled training data
X_train_scaled[:5]

array([[-5.97353422e-04, -1.00933748e+00,  8.08431209e-02,
        -6.60418638e-01,  3.73770647e-01,  8.45307934e-01,
        -3.50759886e-01, -4.80920904e-01,  6.48153603e-01,
        -9.14694849e-01,  1.44458808e+00, -6.17213400e-01],
       [-1.09773647e+00, -7.61501047e-01,  1.07072222e+00,
        -1.57817569e+00, -1.01308206e+00, -9.63486907e-01,
        -3.50759886e-01, -4.80920904e-01,  6.48153603e-01,
        -9.14694849e-01,  1.44458808e+00, -6.17213400e-01],
       [-7.68594737e-01,  3.53762891e-01,  1.07072222e+00,
        -6.60418638e-01, -1.01308206e+00, -9.63486907e-01,
        -6.63492406e-01,  2.07934401e+00, -1.54284416e+00,
         1.09326078e+00, -6.92238854e-01, -6.17213400e-01],
       [-1.10311265e-01, -2.65828186e-01,  1.07072222e+00,
        -6.60418638e-01,  3.73770647e-01, -9.63486907e-01,
        -6.63492406e-01, -4.80920904e-01,  6.48153603e-01,
        -9.14694849e-01,  1.44458808e+00, -6.17213400e-01],
       [ 2.08396697e+00, -1.00933748e+00, -9.0903598

In [9]:
# Check the columns in X_train
X_train.columns


Index(['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
       'JobInvolvement', 'JobLevel', 'YearsSinceLastPromotion',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'MaritalStatus_Married', 'MaritalStatus_Single', 'OverTime_Yes'],
      dtype='object')

In [10]:
# Add the 'Department' column to X_df using .loc
X_df.loc[:, 'Department'] = attrition_df['Department']



<ipython-input-10-f230deae07eb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df.loc[:, 'Department'] = attrition_df['Department']


In [11]:
# Add the 'Department' column to X_train using .loc
X_train.loc[:, 'Department'] = attrition_df.loc[X_train.index, 'Department']


In [12]:
# Check if the 'Department' column has been added to X_df
X_df.columns


Index(['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
       'JobInvolvement', 'JobLevel', 'YearsSinceLastPromotion',
       'BusinessTravel', 'MaritalStatus', 'OverTime', 'Department'],
      dtype='object')

In [13]:
X_train.columns


Index(['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
       'JobInvolvement', 'JobLevel', 'YearsSinceLastPromotion',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'MaritalStatus_Married', 'MaritalStatus_Single', 'OverTime_Yes',
       'Department'],
      dtype='object')

In [14]:
X_test.columns


Index(['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
       'JobInvolvement', 'JobLevel', 'YearsSinceLastPromotion',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'MaritalStatus_Married', 'MaritalStatus_Single', 'OverTime_Yes'],
      dtype='object')

In [15]:
# Add the 'Department' column to X_test using .loc
X_test.loc[:, 'Department'] = attrition_df.loc[X_test.index, 'Department']


In [16]:
from sklearn.preprocessing import OneHotEncoder

In [17]:
# Create a OneHotEncoder for the Department column
encoder_department = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data (only on the Department column)
encoder_department.fit(X_train[['Department']])

# Create two new variables by applying the encoder to the training and testing data
X_train_department = encoder_department.transform(X_train[['Department']])
X_test_department = encoder_department.transform(X_test[['Department']])

# Preview the transformed department data for training and testing sets
X_train_department[:5], X_test_department[:5]


(array([[1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.]]),
 array([[1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.]]))

In [18]:
from sklearn.preprocessing import OneHotEncoder

In [19]:
# Import the OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Attrition column
encoder_attrition = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data (only on the Attrition column)
encoder_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder to the training and testing data
y_train_attrition = encoder_attrition.transform(y_train[['Attrition']])
y_test_attrition = encoder_attrition.transform(y_test[['Attrition']])

# Preview the transformed attrition data for training and testing sets
y_train_attrition[:5], y_test_attrition[:5]



(array([[0.],
        [0.],
        [1.],
        [0.],
        [0.]]),
 array([[1.],
        [1.],
        [1.],
        [0.],
        [0.]]))

## Part 2: Create, Compile, and Train the Model

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [21]:
# Find the number of columns in the X training data
num_columns = X_train.shape[1]
print(f"Number of columns in X_train: {num_columns}")

# Create the input layer with a unique name
model = Sequential()
model.add(Dense(64, input_dim=num_columns, activation='relu', name="input_layer_1_unique"))

# Create two shared layers with unique names
model.add(Dense(32, activation='relu', name="shared_layer_1_unique"))
model.add(Dense(16, activation='relu', name="shared_layer_2_unique"))

Number of columns in X_train: 13


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer for the Department branch
dept_hidden = Dense(16, activation="relu")(shared_layer2)

# Create the output layer for the Department branch
dept_output = Dense(3, activation="softmax", name="department_output")(dept_hidden)



In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer for the Attrition branch
attrition_hidden = Dense(16, activation="relu")(shared_layer2)

# Create the output layer for the Attrition branch
attrition_output = Dense(1, activation="sigmoid", name="attrition_output")(attrition_hidden)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Create the model
model = Model(inputs=input_layer, outputs=[attrition_output, dept_output])

# Compile the model
model.compile(
    loss={
        "attrition_output": "binary_crossentropy",
        "dept_output": "categorical_crossentropy"
    },
    optimizer=Adam(),
    metrics={
        "attrition_output": "accuracy",
        "dept_output": "accuracy"
    }
)

# Summarize the model
model.summary()


In [ ]:
attrition_df.columns


In [ ]:
# 1. First, separate your features and targets from the full dataset
X = attrition_df.drop(['Attrition', 'Department'], axis=1)
y_attrition = attrition_df['Attrition']
y_department = attrition_df['Department']

# 2. Split into training and validation sets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train_attrition, y_val_attrition = train_test_split(X, y_attrition, test_size=0.2, random_state=42)
_, _, y_train_department, y_val_department = train_test_split(X, y_department, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode Attrition (binary classification)
le_attrition = LabelEncoder()
y_train_attrition_enc = le_attrition.fit_transform(y_train_attrition)
y_val_attrition_enc = le_attrition.transform(y_val_attrition)

# Encode Department (multi-class classification)
le_dept = LabelEncoder()
y_train_department_enc = le_dept.fit_transform(y_train_department)
y_val_department_enc = le_dept.transform(y_val_department)

# One-hot encode department for softmax output
y_train_department_ohe = to_categorical(y_train_department_enc)
y_val_department_ohe = to_categorical(y_val_department_enc)


In [ ]:
# Combine the full dataset before splitting to keep feature columns aligned
X_encoded = pd.get_dummies(X)

# Re-split the encoded features using the same random state
X_train, X_val = train_test_split(X_encoded, test_size=0.2, random_state=42)


In [ ]:
# Ensure all data in X is numeric and of float32 type
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')


In [ ]:
print(X_train.dtypes)


In [ ]:
print(type(y_train_attrition_enc), y_train_attrition_enc.dtype)
print(type(y_train_department_ohe), y_train_department_ohe.dtype)


In [ ]:
import numpy as np

# Ensure correct dtypes
y_train_attrition_enc = np.array(y_train_attrition_enc).astype('int32')
y_val_attrition_enc = np.array(y_val_attrition_enc).astype('int32')

y_train_department_ohe = np.array(y_train_department_ohe).astype('float32')
y_val_department_ohe = np.array(y_val_department_ohe).astype('float32')


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Input layer
input_layer = Input(shape=(X_train.shape[1],))

# Shared hidden layers
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)

# Output 1: Binary classification (Attrition)
attrition_output = Dense(1, activation='sigmoid', name='attrition_output')(x)

# Output 2: Multi-class classification (Department)
department_output = Dense(y_train_department_ohe.shape[1], activation='softmax', name='department_output')(x)

# Create the model
model = Model(inputs=input_layer, outputs=[attrition_output, department_output])

# Compile it with two loss functions and appropriate metrics
model.compile(
    optimizer=Adam(),
    loss={
        'attrition_output': 'binary_crossentropy',
        'department_output': 'categorical_crossentropy'
    },
    metrics={
        'attrition_output': 'accuracy',
        'department_output': 'accuracy'
    }
)


In [ ]:
# Train the model
history = model.fit(
    X_train,
    {
        'attrition_output': y_train_attrition_enc,
        'department_output': y_train_department_ohe
    },
    validation_data=(
        X_val,
        {
            'attrition_output': y_val_attrition_enc,
            'department_output': y_val_department_ohe
        }
    ),
    epochs=100,
    batch_size=32
)

In [ ]:
print(X_test.shape)  # Check the number of samples in X_test
print(len(y_test_attrition_enc))  # Check the number of samples in y_test_attrition_enc
print(len(y_test_department_ohe))  # Check the number of samples in y_test_department_ohe


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data again if necessary, making sure the same rows are used for both features and targets
X_train, X_test, y_train_attrition, y_test_attrition, y_train_department, y_test_department = train_test_split(
    X, y_attrition, y_department, test_size=0.2, random_state=42
)


In [ ]:
print(X_test.dtypes)  # Check data types in the feature set
print(type(y_test_attrition_enc), y_test_attrition_enc.shape)  # Check target type and shape for attrition
print(type(y_test_department_ohe), y_test_department_ohe.shape)  # Check target type and shape for department


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Identify categorical columns
categorical_columns = [
    'BusinessTravel', 'EducationField', 'JobRole', 'MaritalStatus', 'OverTime'
]

# Define the column transformer with One-Hot Encoding for categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns),
        ('num', SimpleImputer(strategy='mean'), X_test.select_dtypes(include=['int64', 'float64']).columns)
    ]
)

# Apply the transformations to both the training and test sets
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Check the shape of the transformed data
print(X_train_transformed.shape)
print(X_test_transformed.shape)


In [ ]:
# Evaluate the model with test data
test_loss, attrition_accuracy, department_accuracy = model.evaluate(
    X_test_transformed,
    {
        'attrition_output': y_test_attrition_enc,
        'department_output': y_test_department_ohe
    },
    batch_size=32
)

# Print the results
print(f"Test Loss: {test_loss}")
print(f"Attrition Accuracy: {attrition_accuracy}")
print(f"Department Accuracy: {department_accuracy}")


In [ ]:
# Evaluate the model with test data
test_loss, attrition_loss, attrition_accuracy, department_loss, department_accuracy = model.evaluate(
    X_test_transformed,
    {
        'attrition_output': y_test_attrition_enc,
        'department_output': y_test_department_ohe
    },
    batch_size=32
)

# Print the results
print(f"Test Loss: {test_loss}")
print(f"Attrition Loss: {attrition_loss}")
print(f"Attrition Accuracy: {attrition_accuracy}")
print(f"Department Loss: {department_loss}")
print(f"Department Accuracy: {department_accuracy}")


In [ ]:
# Print the accuracy for both department and attrition
print(f"Attrition Accuracy: {attrition_accuracy * 100:.2f}%")
print(f"Department Accuracy: {department_accuracy * 100:.2f}%")



In [ ]:
# Print the full evaluation result
print(f"Evaluation Result: {test_loss}, {attrition_accuracy}, {department_accuracy}")


In [ ]:
# If using a multi-output model, unpack the results correctly
test_loss, attrition_loss, attrition_accuracy, department_loss, department_accuracy = model.evaluate(
    X_test_transformed,
    {
        'attrition_output': y_test_attrition_enc,
        'department_output': y_test_department_ohe
    },
    batch_size=32
)

# Print the results for each
print(f"Test Loss: {test_loss}")
print(f"Attrition Loss: {attrition_loss}")
print(f"Attrition Accuracy: {attrition_accuracy * 100:.2f}%")
print(f"Department Loss: {department_loss}")
print(f"Department Accuracy: {department_accuracy * 100:.2f}%")


In [ ]:
# Print the raw values of the metrics
print(f"Raw Attrition Accuracy: {attrition_accuracy}")
print(f"Raw Department Accuracy: {department_accuracy}")


In [ ]:
# Check the model's metrics names to confirm the unpacking
print("Model metrics:", model.metrics_names)


In [ ]:
model.compile(
    optimizer=Adam(),
    loss={
        'attrition_output': 'binary_crossentropy',
        'department_output': 'categorical_crossentropy'
    },
    metrics={
        'attrition_output': ['accuracy'],  # Added accuracy here
        'department_output': ['accuracy']  # Added accuracy here
    }
)


In [ ]:
# Reevaluate the model after fixing the metrics
test_loss, attrition_loss, attrition_accuracy, department_loss, department_accuracy = model.evaluate(
    X_test_transformed,
    {
        'attrition_output': y_test_attrition_enc,
        'department_output': y_test_department_ohe
    },
    batch_size=32
)

# Print the results for each
print(f"Test Loss: {test_loss}")
print(f"Attrition Loss: {attrition_loss}")
print(f"Attrition Accuracy: {attrition_accuracy * 100:.2f}%")
print(f"Department Loss: {department_loss}")
print(f"Department Accuracy: {department_accuracy * 100:.2f}%")


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Define input layer
inputs = Input(shape=(X_train_transformed.shape[1],))  # Adjust the shape based on your input data

# Add the hidden layers as needed
x = Dense(64, activation='relu')(inputs)  # Example hidden layer
x = Dense(32, activation='relu')(x)      # Example hidden layer

# Output layer for Attrition (binary classification)
attrition_output = Dense(1, activation='sigmoid', name='attrition_output')(x)

# Output layer for Department (multi-class classification)
department_output = Dense(n_classes, activation='softmax', name='department_output')(x)

# Create the model
model = Model(inputs=inputs, outputs=[attrition_output, department_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'attrition_output': 'binary_crossentropy',
        'department_output': 'categorical_crossentropy'
    },
    metrics={
        'attrition_output': ['accuracy'],
        'department_output': ['accuracy']
    }
)


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.